# Predicting House Prices in King County, Washington
## Linear Regression Model
### by Christine Egan

## Alternative Hypothesis:
Location, size, quality, and age have an association with house prices in King County.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats as stats
from scipy.stats import zscore

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt

%matplotlib inline
from functions1 import std_describe, percent_null_df, obtain_data
from functions1 import idx_select_sort_set, remove_dupes, find_nulls
from functions1 import rm_outliers_by_zscore, corr_heat_map, draw_qqplot
from functions1 import reg_summary, get_residual, validate_reg_assumptions

## 1. Obtaining Data

In [ ]:
data = obtain_data('https://raw.githubusercontent.com/snepaul179/Real_EstateModel-onl01-dtsc-pt-052620/master/kc_house_data.csv')

## 2. Scrubbing Data

### i. Selecting, Sorting, and Setting the Index
Examining the first few rows of the dataframe, we can see that there is column called 'id.' We can assume that each house is given its own unique id number. However, since there is also a date column, we should check for duplicates in both 'id' and 'date' before we set the index to 'id.' In addition, it would be helpful to transform 'date' into a datetime object so we can use it to create other features later on.

### ii. Identifying Null Values
The columns 'view' and 'waterfront' contain very little data. In the 'waterfront' column, we can see that there are only 146 properties out of more than 21,000 properties that can be considered waterfront properties. From this perspective, it seems like an unlikely predictor. Similarly, in 'view', only a small number of properties have ever been viewed. Due to this reasons, we will drop these columns. In 'yr_renovated', nearly 93% of the data is null or 0.0. Due to this, it also seems like a poor choice of a predictor and should be dropped.

### iii. Changing Data Types
In the information above, 'sqft_basement' is described as an 'object' when it is clear that the data in the column should be numeric. First, we will check the values to make sure they are all numeric, then we will change the column to a numeric data type. For some reason, there are a bunch of question marks in this column. Since it is a few hundred rows, we don't want to completely eliminate them from the data set. We will set them to '0'.

In [ ]:
#i. select, sort, and set the index, change the date to a datetime object
indices = ['id', 'date']
data = idx_select_sort_set(data, indices)
data['date'] =  pd.to_datetime(data.date, infer_datetime_format=True)

# ii. identifying null values and taking action
print(find_nulls(data))
data = data.drop(['view', 'waterfront'], axis=1) # only 146 wf properties, very few for view as well

# print(data[data.yr_renovated.notna() & data.yr_renovated != 0].shape) --> only 746 have reno info!
data = data.drop('yr_renovated', axis=1)

# iii. changing data types
print('sqft_basement value counts:\n', data.sqft_basement.value_counts())
data['sqft_basement'] = data.sqft_basement.replace('?', '0')
data['sqft_basement'] = pd.to_numeric(data.sqft_basement, errors='coerce')
data['sqft_basement'] = data.sqft_basement.fillna('0')

## 3. Exploring Data

### i. First Check for Outliers
We can see that the price column has some exetreme outliers. I think that removing some of the most exetreme outliers seem to be in the higher range, however there are some in the lower range as well.

In [ ]:
target = ['price']
drop_cols = ['date', 'lat', 'long', 'zipcode'] # not necessary for histogram
std_describe(data.drop(drop_cols, axis=1)) # getting preliminary information about outliers

In [ ]:
data.drop(drop_cols, axis=1).hist(figsize=(12,12));

### ii. Addressing Outliers by Z-Score
The columns 'price', 'bedrooms', and 'bathrooms' seem to have some outliers with exetreme values on the higher end. I want restrict the data to values that have a z-score with an absolute value of less than 3. This will enable the data to have some degree of variation, without allowing exetreme outliers to strongly influence the data.

In [ ]:
cols = ['price', 'bedrooms', 'bathrooms']
data = rm_outliers_by_zscore(data, cols)

### ii. Creating/Modifying Features
#### How can the features be modified to address location, size, age, quality, and functionality?

a. Binary Features
* In bedrooms and bathrooms, the exetreme outliers have been removed by z-score. While I still want to explore the relationship between the number of bedrooms and bathrooms and the price of a home, I think that adding a column for each possible amount will add unncessary complexity. A soultion is to turn them into binary variables that capture if the number of bathrooms falls beneath the 75th percentile, 'four_plus_bedrooms', and 'three_plus_bathrooms.'
* In grade, outliers were not addressed by z-score, however it should be taken into account that some properties with high grades may have been eliminated when the outliers were removed from 'price.' In this case we will also use the 75th percentile to create a binary variable, 'eight_plus_grade.'
* In'sqft_basement' is exetremely skewed because houses with no basements are represented by '0'. I think one way to still use this feature as a predictor is to work with it as a binary.

b. Reimagining Price
* One column that can help me to build other columns using averages, would be a column called 'price_per_sqft_living', combining 'price' and 'sqft_living.' It would not be wise to use this column as a predictor, but it can be used to create new features.

c. Using Averages
* Using price, I can obtain the average price per zipcode, allowing me to group zipcodes and properties.
* With sqft, I can obtain the average sqft_living per zipcode.
* Combining these two, I can obtain the average price per square foot by zipcode.

d. Determining Age from Dates
* A better way to use the date information provided is to transform it into a feature that expresses the age of the home. For some reason there were 12 values that had -1 as the age of the home. Since it was just a few, they were dropped.

In [ ]:
# a. creating some binary features
data['four_plus_bedrooms'] = data.bedrooms.apply(lambda x: 1 if x >= 4 else 0)
data['three_plus_bathrooms'] = data.bedrooms.apply(lambda x: 1 if x >= 3 else 0)
data['eight_plus_grade'] = data.grade.apply(lambda x: 1 if x >= 8 else 0)
data['basement'] = data.sqft_basement.apply(lambda x: 1 if x > 0 else 0)

# b. creating new feature to describe price per square foot
data['price_per_sqft_living'] = data.price/data.sqft_living

# c. creating another feature for avg price by zip and another feature average sqft by zip
avg_price = data.groupby(['zipcode', 'price']).sum().reset_index().groupby('zipcode').mean().astype(int)
d = avg_price.price.to_dict()
data['zip_avg_price'] = data.zipcode.map(d)

avg_sqft_living = data.groupby(['zipcode', 'sqft_living']).sum().reset_index().groupby('zipcode').mean().astype(int)
d = avg_price.sqft_living.to_dict()
data['zip_avg_sqft'] = data.zipcode.map(d)

# creating another feature by combining the other two
data['zip_avg_pp_sqft'] = round((data.zip_avg_price/data.zip_avg_sqft), 0)

# d. determining age from date information
data['yr_sold'] = pd.DatetimeIndex(data['date']).year
data['age_at_sale'] = data.yr_sold - data.yr_built.astype(int) # age of house on date sold

print(sorted(list(data.age_at_sale.unique()))) # not sure why some are -1
print(data[data.age_at_sale == -1].shape[0]) # how many?
data = data[data.age_at_sale != -1] # eliminated

### iii. Dropping Features
Some of these features don't seem essential to testing my hypothesis.

#### Eliminated
| Feature     | Reason for Elimination |
|:---------   | :-----------|
| date     | Transformed to create 'age_at_sale' using 'yr_built'. |
| yr_renovated | There was a lot of missing data, making it an unsuitable predictor. |
| floors       | This information was redundant, and similar information could be obtained by examining square footage and number of rooms. |
| waterfront   | There was a lot of missing data, making it an unsuitable predictor. |
| view         | There was a lot of missing data, making it an unsuitable predictor. |
| lat          | Not needed for this analysis. |
| long         | Not needed for this analysis |
| sqft_above   | This information was redundant, and similar information could be obtained by examining square footage and number of rooms. |
| sqft_basement| The skew of this information was severe, so it was used to create a more useful feature, then dropped. |
| sqft_lot     | Exetreme outliers made this data fairly unreliable, and efforts to normalize compromised the accuracy of the results. |
| sqft_living15| Demonstrated high correlation with more useful square foot metrics, such as 'sqft_living'.
| sqft_lot15   | The elimination of other features made it irrelevant for analysis.     |
|bedrooms     | Outliers with z-score greater than 3 were removed. Then, it was used to create the feature '4_plus_bedrooms', which contained 0 or 1 to indicate if there were more or less than 4 bedrooms. |
|bathrooms    | Outliers with z-score greater than 3 were removed. Then, it was used to create the feature '3_plus_bathrooms', which contained 0 or 1 to indicate if there were more or less than 3 bathrooms. |
|grade        | It was used to create the feature '8_plus_grade', which contained a 0 or 1 to indicate if the grade was more or less than 8. 
|yr_built     | Transformed to create 'age_at_sale' using 'date'.
|zipcode      | Transformed to create the features 'zip_avg_price', 'zip_avg_sqft', 'zip_avg_pp_sqft' using 'price.' |

#### Final Features
The objective of this model was to test which features have a significant influence on the price of a home in King County. To facilitate this, features were chosen to reflect, the size of the home, the age of the home, the quality of the home, and the location of the home.  

| Feature                 | Indicates:                                 |
|:---------               | :-----------                               |
| 4_plus_bedrooms         |  size                                      |
| 3_plus_bathrooms        |  size                                      |
| sqft_living             |  size                                      |
| basement                |  size                                      |   
| zip_avg_pp_sqft_living  |  a relationship between size and price     |
| zip_avg_sqft            |  a relationship between size and location  |
| age_at_sale             |  age                                       |
| condition               |  age & quality                             |
| 8_plus_grade            |  quality                                   | 

In [ ]:
drop_cols = ['price', 'bedrooms', 'bathrooms', 'grade', 
             'sqft_basement', 'price_per_sqft_living', 
             'sqft_above', 'sqft_basement', 'sqft_lot', 
             'sqft_living15', 'sqft_lot15', 'zip_avg_price',
             'date', 'yr_sold', 'yr_built', 'lat', 'long',
             'zipcode', 'floors']

In [ ]:
t_data = data.copy()

### iv. Multicollinearity Check

In [ ]:
corr_heat_map(data, drop_cols)
plt.title('King County, Washington Housing Data \n Correlation Matrix Heat Map \n')

## 4. Modeling and Interpreting the Data

### i. The Comprehensive Model

In [ ]:
drop_cols.remove('price')
x = t_data.drop(drop_cols, axis=1)
X = x.drop(['price'], axis=1)
y = x['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=44)

validate_reg_assumptions(X, X_train, X_test, y_train, y_test)

### ii. Zipcode Group Model
#### Can we isolate each zipcode group and determine the association with price?

In [ ]:
t_data['zip_avg_pp_sqft_one'] = t_data.zip_avg_pp_sqft.apply(lambda x: 1 if x <= 134 else 0)
t_data['zip_avg_pp_sqft_two'] = t_data.zip_avg_pp_sqft.apply(lambda x: 1 if 134 < x <= 171 else 0)
t_data['zip_avg_pp_sqft_three'] = t_data.zip_avg_pp_sqft.apply(lambda x: 1 if 171 < x <= 210 else 0)
t_data['zip_avg_pp_sqft_four'] = t_data.zip_avg_pp_sqft.apply(lambda x: 1 if 210 < x <= 518 else 0)

In [ ]:
drop_cols.append('zip_avg_pp_sqft')
x = t_data.drop(drop_cols, axis=1)
X = x.drop(['price'], axis=1)
y = x['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=44)

validate_reg_assumptions(X, X_train, X_test, y_train, y_test)

### iii. Modeling by Zipcode
#### Can we improve accuracy by trying one model for each zipcode group, and see what associations emerge for each tier?

In [ ]:
zip_avg_pp_sqft_one =  t_data[t_data.zip_avg_pp_sqft_one == 1]
zip_avg_pp_sqft_two =  t_data[t_data.zip_avg_pp_sqft_two == 1] 
zip_avg_pp_sqft_three = t_data[t_data.zip_avg_pp_sqft_three == 1]
zip_avg_pp_sqft_four = t_data[t_data.zip_avg_pp_sqft_four == 1] 

In [ ]:
x = zip_avg_pp_sqft_one.drop(drop_cols, axis=1)
X = x.drop(['price'], axis=1)
y = x['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=44)

validate_reg_assumptions(X, X_train, X_test, y_train, y_test)

In [ ]:
x = zip_avg_pp_sqft_two.drop(drop_cols, axis=1)
X = x.drop(['price'], axis=1)
y = x['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=44)

validate_reg_assumptions(X, X_train, X_test, y_train, y_test)

In [ ]:
x = zip_avg_pp_sqft_three.drop(drop_cols, axis=1)
X = x.drop(['price'], axis=1)
y = x['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=44)

validate_reg_assumptions(X, X_train, X_test, y_train, y_test)

In [ ]:
x = zip_avg_pp_sqft_four.drop(drop_cols, axis=1)
X = x.drop(['price'], axis=1)
y = x['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=44)

validate_reg_assumptions(X, X_train, X_test, y_train, y_test)